In [1]:
!pip install --upgrade ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 23.6 MB/s eta 0:00:0000:0100:01m
  Attempting unins

In [3]:
# Cell 1: Imports and setup directories
import pandas as pd
import cv2
import os
import shutil
from pathlib import Path

# Create dataset structure
os.makedirs('/kaggle/working/datasets/images/train', exist_ok=True)
os.makedirs('/kaggle/working/datasets/images/test', exist_ok=True)
os.makedirs('/kaggle/working/datasets/labels/train', exist_ok=True)
os.makedirs('/kaggle/working/datasets/labels/test', exist_ok=True)

print("Directories created.")

Directories created.


In [5]:
# Cell 2: Copy images to dataset structure (instead of symlinking)
train_input_dir = '/kaggle/input/qrcodes/QR_Dataset/train_images'
test_input_dir = '/kaggle/input/qrcodes/QR_Dataset/test_images'
train_output_dir = '/kaggle/working/datasets/images/train'
test_output_dir = '/kaggle/working/datasets/images/test'

# Copy train images
for i in range(1, 201):
    src = f'{train_input_dir}/img{i:03d}.jpg'
    dst = f'{train_output_dir}/img{i:03d}.jpg'
    if os.path.exists(src):
        shutil.copy2(src, dst)  # Copy file with metadata
    else:
        print(f"Warning: {src} not found.")

# Copy test images
for i in range(201, 251):
    src = f'{test_input_dir}/img{i:03d}.jpg'
    dst = f'{test_output_dir}/img{i:03d}.jpg'
    if os.path.exists(src):
        shutil.copy2(src, dst)
    else:
        print(f"Warning: {src} not found.")

print("Images copied.")

Images copied.


In [6]:
# Cell 3: Load CSV and generate YOLO-format labels
df = pd.read_csv('/kaggle/input/annoted/combined.csv')
df.columns = ['image_id', 'x_min', 'y_min', 'x_max', 'y_max']  # Verify CSV headers

grouped = df.groupby('image_id')

for img_id, group in grouped:
    img_num = int(img_id[3:-4])  # e.g., 'img001.jpg' -> 1
    if 1 <= img_num <= 200:
        label_dir = '/kaggle/working/datasets/labels/train'
        img_base_path = '/kaggle/working/datasets/images/train'
    elif 201 <= img_num <= 250:
        label_dir = '/kaggle/working/datasets/labels/test'
        img_base_path = '/kaggle/working/datasets/images/test'
    else:
        print(f"Skipping unknown image: {img_id}")
        continue
    
    img_path = os.path.join(img_base_path, img_id)
    if not os.path.exists(img_path):
        print(f"Image not found: {img_path}")
        continue
    
    img = cv2.imread(img_path)
    if img is None:
        print(f"Could not load image: {img_path}")
        continue
    
    h, w = img.shape[:2]
    label_filename = f"{img_id[:-4]}.txt"
    label_path = os.path.join(label_dir, label_filename)
    
    with open(label_path, 'w') as f:
        for _, row in group.iterrows():
            xmin = row['x_min']
            ymin = row['y_min']
            xmax = row['x_max']
            ymax = row['y_max']
            
            # Convert to YOLO format: class x_center y_center width height (normalized)
            x_center = ((xmin + xmax) / 2) / w
            y_center = ((ymin + ymax) / 2) / h
            bbox_width = (xmax - xmin) / w
            bbox_height = (ymax - ymin) / h
            
            f.write(f"0 {x_center:.6f} {y_center:.6f} {bbox_width:.6f} {bbox_height:.6f}\n")
    
    print(f"Generated labels for {img_id}")

print("Labels generated.")

Generated labels for img001.jpg
Generated labels for img002.jpg
Generated labels for img003.jpg
Generated labels for img004.jpg
Generated labels for img005.jpg
Generated labels for img006.jpg
Generated labels for img007.jpg
Generated labels for img008.jpg
Generated labels for img009.jpg
Generated labels for img010.jpg
Generated labels for img011.jpg
Generated labels for img012.jpg
Generated labels for img013.jpg
Generated labels for img014.jpg
Generated labels for img015.jpg
Generated labels for img016.jpg
Generated labels for img017.jpg
Generated labels for img018.jpg
Generated labels for img019.jpg
Generated labels for img020.jpg
Generated labels for img021.jpg
Generated labels for img022.jpg
Generated labels for img023.jpg
Generated labels for img024.jpg
Generated labels for img025.jpg
Generated labels for img026.jpg
Generated labels for img027.jpg
Generated labels for img028.jpg
Generated labels for img029.jpg
Generated labels for img030.jpg
Generated labels for img031.jpg
Generate

In [7]:
# Cell 4: Create data.yaml configuration file
yaml_content = """
path: /kaggle/working/datasets
train: images/train
val: images/test
nc: 1
names: ['QR']
"""

with open('/kaggle/working/data1.yaml', 'w') as f:
    f.write(yaml_content)

print("data.yaml created. Contents:")
!cat /kaggle/working/data1.yaml

data.yaml created. Contents:

path: /kaggle/working/datasets
train: images/train
val: images/test
nc: 1
names: ['QR']


In [8]:
# Cell 5: Train the YOLOv12 model
from ultralytics import YOLO

# Load YOLOv12 small model
model = YOLO('yolo12s.pt')  # Changed to small model as per your output

# Train the model
results = model.train(
    data='/kaggle/working/data1.yaml',
    epochs=50,
    imgsz=640,
    batch=16,
    workers=0,     # Set to 0 to avoid issues with Kaggle's environment
    project='/kaggle/working/yolo_runs',
    name='qr_detection',
    patience=100,  # Prevent early stopping for small dataset
    exist_ok=True  # Allow overwriting runs
)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.204 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/data1.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=Fals

/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all         50        180      0.989      0.994      0.994      0.607
Speed: 0.2ms preprocess, 8.2ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to /kaggle/working/yolo_runs/qr_detection


In [9]:
# Cell 6: Validate the trained model for testing accuracy
from ultralytics import YOLO
import numpy as np

# Load the trained model
model = YOLO('/kaggle/working/yolo_runs/qr_detection/weights/best.pt')

# Run validation on the test set
metrics = model.val(data='/kaggle/working/data1.yaml')

# Calculate F1 score
precision = metrics.box.mp
recall = metrics.box.mr
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

# Access confusion matrix to compute TP, FP, FN (and TN if applicable)
if hasattr(metrics, 'confusion_matrix'):
    cm = metrics.confusion_matrix.matrix  # Confusion matrix as numpy array
    # Assuming binary detection (e.g., QR code vs. background) or multi-class summed
    TP = np.diag(cm).sum()  # True Positives: sum of diagonal (correct predictions)
    FP = cm.sum(axis=0)[:-1].sum() - TP  # False Positives: predicted positive but wrong
    FN = cm.sum(axis=1)[:-1].sum() - TP  # False Negatives: missed positives
    TN = cm[-1, -1] if cm.shape[0] > 1 else 0  # True Negatives: background correctly not detected (if applicable)

    # Calculate accuracy: (TP + TN) / (TP + TN + FP + FN)
    total = TP + TN + FP + FN
    accuracy = (TP + TN) / total if total > 0 else 0
else:
    # Fallback: approximate accuracy using TP and FP (if TN not available)
    TP = metrics.box.tp.sum() if hasattr(metrics.box, 'tp') else 0
    FP = metrics.box.fp.sum() if hasattr(metrics.box, 'fp') else 0
    FN = metrics.box.fn.sum() if hasattr(metrics.box, 'fn') else 0
    TN = 0  # TN often not used in object detection
    total = TP + TN + FP + FN
    accuracy = (TP + TN) / total if total > 0 else 0

# Print metrics
print(f"mAP50-95: {metrics.box.map:.4f}")
print(f"mAP50: {metrics.box.map50:.4f}")
print(f"mAP75: {metrics.box.map75:.4f}")
print(f"Precision: {metrics.box.mp:.4f}")
print(f"Recall: {metrics.box.mr:.4f}")
print(f"F1 Score: {f1_score:.4f}")
print(f"Accuracy (TP + TN) / (TP + TN + FP + FN): {accuracy:.4f}")
print(f"Per-class mAP50-95: {metrics.box.maps}")

Ultralytics 8.3.204 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv12s summary (fused): 159 layers, 9,231,267 parameters, 0 gradients, 21.2 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 4720.3±885.8 MB/s, size: 3501.9 KB)
val: Scanning /kaggle/working/datasets/labels/test.cache... 50 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 50/50 86.2Kit/s 0.0s
val: /kaggle/working/datasets/images/test/img201.jpg: corrupt JPEG restored and saved
val: /kaggle/working/datasets/images/test/img202.jpg: corrupt JPEG restored and saved
val: /kaggle/working/datasets/images/test/img203.jpg: corrupt JPEG restored and saved
val: /kaggle/working/datasets/images/test/img204.jpg: corrupt JPEG restored and saved
val: /kaggle/working/datasets/images/test/img205.jpg: corrupt JPEG restored and saved
val: /kaggle/working/datasets/images/test/img206.jpg: corrupt JPEG restored and saved
val: /kaggle/working/datasets/images/test/img207.jpg: corrupt JPEG restored and saved
val: /ka

/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all         50        180      0.989      0.994      0.994      0.607
Speed: 2.7ms preprocess, 16.6ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to /kaggle/working/runs/detect/val
mAP50-95: 0.6074
mAP50: 0.9942
mAP75: 0.6590
Precision: 0.9889
Recall: 0.9942
F1 Score: 0.9916
Accuracy (TP + TN) / (TP + TN + FP + FN): 0.9624
Per-class mAP50-95: [    0.60743]


In [10]:
import os
import pandas as pd
from ultralytics import YOLO

# Load the trained model
model = YOLO('/kaggle/working/yolo_runs/qr_detection/weights/best.pt')

# Load the annotations CSV
df = pd.read_csv('/kaggle/input/annoted/combined.csv')
df.columns = ['image_id', 'x_min', 'y_min', 'x_max', 'y_max']

# Filter for test images (img201.jpg to img250.jpg)
test_image_ids = [f'img{i:03d}.jpg' for i in range(201, 251)]
df_test = df[df['image_id'].isin(test_image_ids)]

# Count ground truth QR codes per test image
ground_truth_counts = df_test.groupby('image_id').size().reset_index(name='ground_truth_qr_codes')

# Predict QR codes on test images
test_images_dir = '/kaggle/working/datasets/images/test'
results = model.predict(
    source=test_images_dir,
    save=True,
    conf=0.25,
    iou=0.45,
    project='/kaggle/working/yolo_runs',
    name='qr_detection_predict'
)

# Count predicted QR codes per image
predicted_counts = []
for result in results:
    img_path = result.path
    img_id = os.path.basename(img_path)
    if img_id in test_image_ids:  # Only include test images
        num_detections = len(result.boxes)
        predicted_counts.append({'image_id': img_id, 'predicted_qr_codes': num_detections})

predicted_df = pd.DataFrame(predicted_counts)

# Merge ground truth and predicted counts
report_df = ground_truth_counts.merge(predicted_df, on='image_id', how='left')
report_df['predicted_qr_codes'] = report_df['predicted_qr_codes'].fillna(0).astype(int)

# Calculate sums for test set
total_ground_truth = report_df['ground_truth_qr_codes'].sum()
total_predicted = report_df['predicted_qr_codes'].sum()

# Print sums only
print("\nQR Code Detection Report (Test Set, img201.jpg to img250.jpg):")
print(f"Total Ground Truth QR Codes: {total_ground_truth}")
print(f"Total Predicted QR Codes: {total_predicted}")

# Save report to a CSV file
report_df.to_csv('/kaggle/working/yolo_runs/qr_detection_predict/qr_code_report.csv', index=False)
print("\nReport saved to /kaggle/working/yolo_runs/qr_detection_predict/qr_code_report.csv")


image 1/50 /kaggle/working/datasets/images/test/img201.jpg: 640x384 1 QR, 56.1ms
image 2/50 /kaggle/working/datasets/images/test/img202.jpg: 384x640 1 QR, 55.0ms
image 3/50 /kaggle/working/datasets/images/test/img203.jpg: 640x384 6 QRs, 15.9ms
image 4/50 /kaggle/working/datasets/images/test/img204.jpg: 640x384 5 QRs, 14.7ms
image 5/50 /kaggle/working/datasets/images/test/img205.jpg: 640x384 5 QRs, 14.6ms
image 6/50 /kaggle/working/datasets/images/test/img206.jpg: 640x384 4 QRs, 15.7ms
image 7/50 /kaggle/working/datasets/images/test/img207.jpg: 640x384 6 QRs, 15.0ms
image 8/50 /kaggle/working/datasets/images/test/img208.jpg: 640x384 4 QRs, 14.8ms
image 9/50 /kaggle/working/datasets/images/test/img209.jpg: 640x384 6 QRs, 16.5ms
image 10/50 /kaggle/working/datasets/images/test/img210.jpg: 640x384 7 QRs, 14.9ms
image 11/50 /kaggle/working/datasets/images/test/img211.jpg: 640x384 1 QR, 15.4ms
image 12/50 /kaggle/working/datasets/images/test/img212.jpg: 640x384 1 QR, 15.3ms
image 13/50 /kag

In [ ]:
import shutil
import os

# Define the folder or file to zip (adjust paths if needed)
folder_to_zip = '/kaggle/working/'  

# Create a zip file of the folder
zip_filename = 'modeltrain.zip'  
shutil.make_archive(zip_filename, 'zip', folder_to_zip)

print(f"Zip file created: /kaggle/working/{zip_filename}")
print("Now go to the Output tab in the right sidebar, find the zip file, and download it using the three dots menu.")